In [1]:
latest_gameweek = 34

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

import gradio as gr
from scipy.ndimage import gaussian_filter

import plotly.graph_objects as go
import seaborn as sns
sns.set_style("darkgrid")


/opt/anaconda3/envs/fpl/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data

In [3]:
# POSITION MAPPING
position_dict = {1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}

In [4]:
filepath = Path(f'../data/predictions/gameweek{latest_gameweek}.csv')
projections = pd.read_csv(filepath, index_col=0)
projections['position'] = projections['element_type'].map(position_dict)
display(projections.head())
display(projections.shape)

,name,element_type,home,opponent_xG_ewm_5,opponent_xG_ewm_10,opponent_xG_ewm_20,opponent_xG_ewm_40,opponent_xGA_ewm_5,opponent_xGA_ewm_10,opponent_xGA_ewm_20,...,gameweek_xA_expanding_per90,gameweek_xGA_expanding_per90,gameweek_xPoints_expanding_per90,xG_overperformance,team_name,opponent_team,date,gameweek,expected_points,position
0,Aaron Cresswell,2.0,1,1.416324,1.467521,1.565065,1.629423,1.561450,1.626832,1.651645,...,0.145138,1.340456,3.600352,1.818182,West Ham,Tottenham,2025-05-04T13:00:00Z,35,2.423078,DEF
1,Aaron Cresswell,2.0,0,1.339371,1.284045,1.332678,1.421138,1.007910,1.243279,1.418048,...,0.145138,1.340456,3.600352,1.818182,West Ham,Manchester Utd,2025-05-11T13:15:00Z,36,2.202546,DEF
2,Aaron Cresswell,2.0,1,0.919345,1.077068,1.176876,1.208376,1.692386,1.520278,1.405347,...,0.145138,1.340456,3.600352,1.818182,West Ham,Nottingham Forest,2025-05-18T13:15:00Z,37,2.723390,DEF
3,Aaron Cresswell,2.0,0,0.689192,0.817924,0.870505,0.887981,2.170316,2.063560,2.024020,...,0.145138,1.340456,3.600352,1.818182,West Ham,Ipswich Town,2025-05-25T15:00:00Z,38,2.784452,DEF
4,Aaron Ramsdale,1.0,0,0.726613,0.759188,0.897113,1.048064,2.283768,2.092293,1.964247,...,0.001652,1.609457,3.906546,NaN,Southampton,Leicester City,2025-05-03T14:00:00Z,35,3.574105,GK


(2220, 114)

In [5]:
if latest_gameweek>0:
    filepath = Path('../data/fpl_df.csv')
elif latest_gameweek==0:
    filepath = Path('../data/fpl_df_preseason.csv')
fpl_df = pd.read_csv(filepath, index_col=0, low_memory=False)
fpl_df = fpl_df[fpl_df.season=='24-25']
display(fpl_df.head())
display(fpl_df.shape)

,assists,bonus,bps,clean_sheets,corners_and_indirect_freekicks_order,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,dreamteam_count,...,has_temporary_code,opta_code,mng_win,mng_draw,mng_loss,mng_underdog_win,mng_underdog_draw,mng_clean_sheets,mng_goals_scored,birth_date
28173,0.0,0.0,1.0,0.0,NaN,0.8,230.0,23.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28174,0.0,0.0,22.0,1.0,NaN,1.4,193.0,52.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28175,1.0,3.0,48.0,1.0,NaN,24.1,37.0,2.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28176,0.0,0.0,2.0,0.0,NaN,0.8,229.0,71.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28177,0.0,0.0,16.0,1.0,NaN,36.7,19.0,17.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(10157, 224)

In [6]:
[col for col in fpl_df.columns if 'oints' in col]

['event_points',
 'points_per_game',
 'total_points',
 'gameweek_xPoints',
 'event_points_ewm_5',
 'gameweek_xPoints_ewm_5',
 'event_points_ewm_10',
 'gameweek_xPoints_ewm_10',
 'event_points_ewm_20',
 'gameweek_xPoints_ewm_20',
 'event_points_ewm_40',
 'gameweek_xPoints_ewm_40',
 'event_points_expanding',
 'gameweek_xPoints_expanding',
 'event_points_expanding_per90',
 'gameweek_xPoints_expanding_per90',
 'points_per_game_rank',
 'points_per_game_rank_type']

In [7]:
df = fpl_df.groupby('name').last().reset_index()[['name', 'team_name', 'element_type', 'now_cost', 
                            'gameweek_minutes_ewm_20', 'points_per_game', 'total_points', 
                            'gameweek_xPoints_ewm_5', 'gameweek_xPoints_ewm_10', 'gameweek_xPoints_ewm_20', 'gameweek_xPoints_ewm_40']]
df['games_played'] = np.round(np.where(df['points_per_game']!=0, df['total_points'] / df['points_per_game'], 0),0)
df['price'] = df['now_cost'] / 10.0
df['value'] = df['gameweek_xPoints_ewm_20'] / df['price']
df['value_points'] = np.sqrt( df['gameweek_xPoints_ewm_20'] *  df['value'])

# EXPECTED POINTS
expected_points_next_10gw = (projections[projections.gameweek.isin( np.arange(latest_gameweek+1, latest_gameweek+11, 1) )]
 .groupby('name')
 .sum()
 )[['expected_points']].reset_index().rename(columns={'expected_points':'expected_points_next_10_GW'})

expected_points_next_5gw = (projections[projections.gameweek.isin( np.arange(latest_gameweek+1, latest_gameweek+6, 1) )]
 .groupby('name')
 .sum()
 )[['expected_points']].reset_index().rename(columns={'expected_points':'expected_points_next_5_GW'})

df = df.merge(expected_points_next_10gw, on='name', how='left')
df = df.merge(expected_points_next_5gw, on='name', how='left')

# POSITION MAPPING
df['position'] = df['element_type'].map(position_dict)

# SEASON TOTALS AND MEANS (XPOINTS AND POINTS)

season_total_xPoints = \
    (fpl_df
     .groupby('name')[['gameweek_xPoints']]
     .sum()
     .rename(columns={'gameweek_xPoints':'season_total_xpoints'})
     .reset_index())

season_total_points = \
    (fpl_df
     .groupby('name')[['event_points']]
     .sum()
     .rename(columns={'event_points':'season_total_points'})
     .reset_index())

season_mean_xPoints = \
    (fpl_df
     .groupby('name')[['gameweek_xPoints']]
     .mean()
     .rename(columns={'gameweek_xPoints':'season_mean_xpoints'})
     .reset_index())

season_mean_points = \
    (fpl_df
     .groupby('name')[['event_points']]
     .mean()
     .rename(columns={'event_points':'season_mean_points'})
     .reset_index())

df = df.merge(season_total_xPoints, on='name', how='left')
df = df.merge(season_total_points, on='name', how='left')
df = df.merge(season_mean_xPoints, on='name', how='left')
df = df.merge(season_mean_points, on='name', how='left')

display(df)

,name,team_name,element_type,now_cost,gameweek_minutes_ewm_20,points_per_game,total_points,gameweek_xPoints_ewm_5,gameweek_xPoints_ewm_10,gameweek_xPoints_ewm_20,...,price,value,value_points,expected_points_next_10_GW,expected_points_next_5_GW,position,season_total_xpoints,season_total_points,season_mean_xpoints,season_mean_points
0,Aaron Cresswell,West Ham,2.0,39.0,52.512039,1.9,27.0,2.803893,2.411046,2.321357,...,3.9,0.595220,1.175465,10.133466,10.133466,DEF,29.055897,27.0,2.075421,1.928571
1,Aaron Ramsdale,Southampton,1.0,44.0,90.000000,3.2,82.0,3.895245,3.692196,3.699326,...,4.4,0.840756,1.763584,12.926561,12.926561,GK,88.574535,82.0,3.406713,3.153846
2,Aaron Wan-Bissaka,West Ham,2.0,44.0,87.907608,2.9,93.0,3.205261,3.196648,3.123114,...,4.4,0.709799,1.488886,12.563993,12.563993,DEF,99.918541,93.0,3.122454,2.906250
3,Abdoulaye Doucouré,Everton,3.0,51.0,81.928734,2.8,85.0,3.444912,3.257138,3.140105,...,5.1,0.615707,1.390462,14.067145,14.067145,MID,80.052240,78.0,2.859009,2.785714
4,Abdukodir Khusanov,Manchester City,2.0,50.0,84.596259,2.3,14.0,2.499536,2.610090,2.643803,...,5.0,0.528761,1.182345,10.685989,10.685989,DEF,15.988129,14.0,2.664688,2.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,Yunus Emre Konak,Brentford,3.0,44.0,5.980804,1.0,8.0,1.065160,1.081398,1.088202,...,4.4,0.247319,0.518780,6.853062,6.853062,MID,8.751356,8.0,1.093919,1.000000
551,Yves Bissouma,Tottenham,3.0,48.0,54.327036,1.9,45.0,1.003498,1.328950,1.616564,...,4.8,0.336784,0.737857,7.436040,7.436040,MID,40.113270,45.0,1.671386,1.875000
552,Zain Silcott-Duberry,Bournemouth,3.0,45.0,1.000000,1.0,1.0,1.000257,1.000257,1.000257,...,4.5,0.222279,0.471526,6.559092,6.559092,MID,1.000257,1.0,1.000257,1.000000
553,Álex Moreno Lopera,Nottingham Forest,2.0,42.0,55.886316,2.3,34.0,2.138889,2.296205,2.340063,...,4.2,0.557158,1.141834,9.101591,9.101591,DEF,40.161542,34.0,2.677436,2.266667


# Gradio functions

In [8]:
def plot_points_and_value(positions, teams, show_names, max_price, x_axis_feature, y_axis_feature):
    
    fig = go.Figure()

    # aux df for manipulation
    df_out = df[df.position.isin(positions)].copy()
    # choose only given teams
    if "Select All" not in teams:
        df_out = df_out[df_out.team_name.isin(teams)]

    # drop players above max_price
    df_out = df_out[df_out.price<=max_price]

    # CREATE FIGURE
    fig.add_trace(
            go.Scatter(
                x=df_out[x_axis_feature],
                y=df_out[y_axis_feature],
                mode="markers+text",
                hovertext=df_out['name'].values,
                showlegend=False,
                ),
        )

    # add player names as visible
    if show_names:
        fig.update_traces(
            text = df_out['name'].values,
            textposition='top center',
            )

    if (x_axis_feature=='season_mean_xpoints') & (y_axis_feature=='season_mean_points'):
        fig.add_trace(
            go.Scatter(
                x=np.linspace(0,9),
                y=np.linspace(0,9),
                mode='lines',
            ),
        ) 

    # styling
    fig.update_layout(
        #title="",
        template='plotly_dark',
        xaxis_title=x_axis_feature,
        yaxis_title=y_axis_feature,
        #showlegend=True
    )

    return fig

In [9]:
def top_weekly_projections(positions, teams, gameweek):
    
    fig = go.Figure()

    # aux df for manipulation
    projections_out = projections[projections.position.isin(positions)].copy()
    # choose only given teams
    if "Select All" not in teams:
        projections_out = projections_out[projections_out.team_name.isin(teams)]
    # choose gameweek
    projections_out = projections_out[projections_out.gameweek==int(gameweek)]

    top_20 = projections_out.groupby('name').sum().sort_values(by='expected_points').reset_index().tail(20)

    # CREATE FIGURE
    fig.add_trace(
        go.Bar(
            x=top_20.expected_points,
            #y=np.arange(top_20.shape[0],0,-1), 
            y=top_20.name, 
            text=np.round(top_20.expected_points,2),
            textposition='outside',
            width=0.75,
            orientation='h'
        ),
    ) 

    # styling
    fig.update_layout(
        #title="",
        template='plotly_dark',
        xaxis_title='gameweek expected points',
        #yaxis_title='value',
        #showlegend=True
    )

    return fig

In [10]:
def player_points(players):

    marker_colors = ['red', 'blue']
    colors = ['rgba(255, 0, 0, 0.5)','rgba(0, 0, 255, 0.5)']
    my_df = pd.DataFrame()

    fig = go.Figure()
    for count, player in enumerate(players):

        aux = df.loc[df['name']==player, ['name', 'team_name', 'position', 'price', 
                            'season_mean_xpoints', 'points_per_game', 
                            'season_total_xpoints', 'total_points', 
                            'gameweek_xPoints_ewm_5', 'gameweek_xPoints_ewm_40', ]].copy()

        my_past_data = fpl_df[fpl_df['name']==player].sort_values(by='gameweek')
        my_projections = projections[projections['name']==player].sort_values(by='gameweek')
        
        x_past = list(np.unique(my_past_data['gameweek']))
        x_future = list(np.unique(my_projections['gameweek']))
        my_x = x_past + x_future

        y_past = list(my_past_data.groupby('gameweek').sum()['gameweek_xPoints'])
        y_future = list(my_projections.groupby('gameweek').sum()['expected_points'])
        my_y = y_past + y_future
        my_y_filtered = gaussian_filter(y_past + y_future, sigma=2, mode='nearest')

        new_cols = [f'xPoints_gameweek_{i}' for i in x_future]
        aux[new_cols] = y_future
        my_df = pd.concat([my_df, aux])

        fig.add_trace(
            go.Scatter(
                x=my_x,
                y=my_y_filtered,
                mode="markers+lines",
                marker=dict(color=marker_colors[count]),  
                line=dict(color=marker_colors[count], width=3),  
                fill='tozeroy',    
                fillcolor=colors[count],   
                name=player,            
                showlegend=True,
                ),
        )

        if len(players)==1:
            fig.add_trace(
                go.Scatter(
                    x=my_x,
                    y=my_y,
                    mode="markers",
                    marker=dict(color='white'),
                    name=player,            
                    showlegend=False,
                    ),
            )

    fig.add_vline(x=latest_gameweek+0.5,)

    fig.update_layout(
        #title="",
        template='plotly_dark',
        xaxis_title="gameweek",
        yaxis_title='expected points',
        #showlegend=True
    )

    return fig, my_df

# Demo

In [11]:
position_list = ['GK', 'DEF', 'MID', 'FWD']
team_name_list = ["Select All"]
team_name_list += list(np.sort(df.team_name.unique()))
gameweek_list = [str(x) for x in np.arange(latest_gameweek+1, latest_gameweek+11,)]
features = ["gameweek_xPoints_ewm_5", "gameweek_xPoints_ewm_20", 'expected_points_next_10_GW', 'value', 
            'season_total_xpoints', 'season_total_points', 'season_mean_xpoints', 'season_mean_points']
x_axis_feature = features
y_axis_feature = features
minimum_price = df['price'].min()
maximum_price = df['price'].max()

In [12]:
if latest_gameweek>0:
    scatter_demo = gr.Interface(
        plot_points_and_value,
        [
            gr.CheckboxGroup(position_list, label="POSITION", value=position_list),
            gr.Dropdown(team_name_list, label="TEAM", multiselect=True, value="Select All"),
            gr.Checkbox('Show player names', value=False),
            gr.Slider(minimum_price, maximum_price, value=maximum_price, info='Choose maximum allowed player value.'),
            gr.Dropdown(x_axis_feature, label="x-axis", value='season_mean_xpoints'),
            gr.Dropdown(y_axis_feature, label="y-axis", value = 'season_mean_points'),
        ],
        gr.Plot(),
    )

In [13]:
weekly_top_players_demo = gr.Interface(
    top_weekly_projections,
    [
        gr.CheckboxGroup(position_list, label="POSITION", value=position_list),
        gr.Dropdown(team_name_list, label="TEAM", multiselect=True, value="Select All"),
        gr.Dropdown(gameweek_list, label="GAMEWEEK", value=str(latest_gameweek+1)),
    ],
    gr.Plot(),
)

In [14]:
player_name_list = list(projections.name.unique())

In [15]:
# player_points_demo = gr.Interface(
#     fn=player_points,
#     inputs=[
#         gr.Dropdown(player_name_list, value='Erling Haaland', multiselect=True, max_choices=2, label='Choose 1 or 2 players')
#     ],
#     outputs=[gr.Plot(), gr.DataFrame()]
# )

In [16]:
with gr.Blocks() as player_points_demo:
    with gr.Column():
        choose_players = gr.Dropdown(
            player_name_list, 
            value='Erling Haaland', 
            multiselect=True, 
            max_choices=2, 
            label='Choose 1 or 2 players'
            )
        submit_button = gr.Button("Submit")
        player_plot = gr.Plot()
        player_data = gr.DataFrame()
    
    submit_button.click(fn=player_points, inputs=choose_players, outputs=[player_plot, player_data])

In [17]:
if latest_gameweek>0:
    full_demo = gr.TabbedInterface(
        [scatter_demo, weekly_top_players_demo, player_points_demo],
        ['Scatter plots', 'Gameweek projections top 20', 'Player xPoints and projections']
    ).launch()
elif latest_gameweek==0:
    full_demo = gr.TabbedInterface(
        [weekly_top_players_demo, player_points_demo],
        ['Gameweek projections top 20', 'Player xPoints and projections']
    ).launch()
else:
    print("Choose valid 'latest_gameweek'!")

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


# DEV STUFF